In [1]:
import os
from enum import Enum
import gzip
import time

import numpy as np
from scipy.sparse import dok_matrix, csr_matrix
import tensorflow as tf

# Attalos Imports
import attalos.util.log.log as l
from attalos.dataset.dataset import Dataset
from attalos.evaluation.evaluation import Evaluation

# Local models
from mse import MSEModel
from negsampling import NegSamplingModel
from fast0tag import FastZeroTagModel

In [2]:
# Setup global objects
logger = l.getLogger(__name__)

In [3]:
from attalos.imgtxt_algorithms.regress2sum.multihot import MultihotModel
from attalos.imgtxt_algorithms.regress2sum.naivesum import NaiveSumModel
from attalos.imgtxt_algorithms.regress2sum.wdv import WDVModel
from attalos.dataset.wordvectors.glove import GloveWrapper

In [4]:
# Temp object using duck typing to replace command line arguments
args = lambda: None
#args.image_feature_file_train = "/local_data/teams/attalos/features/image/espgame_train_20160823_inception.hdf5"
#args.text_feature_file_train = "/local_data/teams/attalos/features/text/espgame_train_20160823_text.json.gz"
#args.image_feature_file_test = "/local_data/teams/attalos/features/image/espgame_test_20160823_inception.hdf5"
#args.text_feature_file_test = "/local_data/teams/attalos/features/text/espgame_test_20160823_text.json.gz"
args.image_feature_file_train = "/local_data/teams/attalos/features/image/iaprtc_train_20160816_inception.hdf5"
args.text_feature_file_train = "/local_data/teams/attalos/features/text/iaprtc_train_20160816_text.json.gz"
args.image_feature_file_test = "/local_data/teams/attalos/features/image/iaprtc_test_20160816_inception.hdf5"
args.text_feature_file_test = "/local_data/teams/attalos/features/text/iaprtc_test_20160816_text.json.gz"
args.word_vector_file = "/local_data/kylez/glove.6B.200d.txt"
args.word_vector_type = "glove"
args.model_type = "wdv"
args.cross_eval = False
args.in_memory = True
args.model_input_path = None
args.model_output_path = None
args.num_epochs = 400
args.batch_size = 100
args.learning_rate = 0.0001

In [5]:
class WordVectorTypes(Enum):
    w2v = 1
    glove = 2

In [6]:
class ModelTypes(Enum):
    mse = 1
    negsampling = 2
    fast0tag = 3
    multihot = MultihotModel
    naivesum = NaiveSumModel
    wdv = WDVModel

In [7]:
def train_batch(sess, model, batch):
    train_x, train_y = batch
    training_loss = model.fit(sess, train_x, train_y)
    return training_loss

In [8]:
def train_epoch(sess, model, train_dataset, batch_size):
    training_losses = []
    for cur_batch_num, batch in enumerate(model.to_batches(train_dataset, batch_size)):
        training_loss = train_batch(sess, model, batch)
        training_losses.append(training_loss)
    avg_training_loss = sum(training_losses) / float(len(training_losses))
    return avg_training_loss

In [9]:
def train(sess, model, num_epochs, train_dataset, batch_size, epoch_verbosity_rate=10):
    for cur_epoch in xrange(num_epochs):
        verbose = cur_epoch % epoch_verbosity_rate == 0
        avg_training_loss = train_epoch(sess, model, train_dataset, batch_size)
        if verbose:
            logger.info("Finished epoch %s. (Avg. training loss: %s)" % (cur_epoch, avg_training_loss))

In [10]:
def load_wv_model(word_vector_file, word_vector_type):
    if args.word_vector_type == WordVectorTypes.glove.name:
        from glove import Glove
        glove_model = Glove.load_stanford(word_vector_file)
        wv_model = GloveWrapper(glove_model)
    else: #args.word_vector_type == WordVectorTypes.w2v.name:
        import word2vec
        w2v_model = word2vec.load(word_vector_file)
        wv_model = W2VWrapper(w2v_model)
    return wv_model

In [11]:
logger.info("Parsing train and test datasets.")
train_dataset = Dataset(args.image_feature_file_train, args.text_feature_file_train, load_image_feats_in_mem=args.in_memory)
test_dataset = Dataset(args.image_feature_file_test, args.text_feature_file_test)

logger.info("Reading word vectors from file.")
wv_model = load_wv_model(args.word_vector_file, args.word_vector_type)

[2016-09-09 19:37:07,239] [INFO] Parsing train and test datasets.
[2016-09-09 19:37:07,557] [INFO] Reading word vectors from file.


In [13]:
#sess.close()
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

In [14]:
model_cls = ModelTypes[args.model_type].value
logger.info("Selecting model class: %s" % model_cls.__name__)
#datasets = [train_dataset] if args.cross_eval else [train_dataset, test_dataset]
model = model_cls(wv_model, train_dataset=train_dataset, test_dataset=test_dataset, **vars(args))
model.initialize_model(sess)

[2016-09-09 19:42:14,013] [INFO] Selecting model class: WDVModel
[2016-09-09 19:42:18,153] [DEBUG] Constructing w1.
[2016-09-09 19:42:25,909] [DEBUG] No specified vocab2. Setting w2 = w1.
[2016-09-09 19:42:25,912] [DEBUG] Multiplying w1 and w2.
[2016-09-09 19:42:25,916] [DEBUG] Applying preprocess_fn.
[2016-09-09 19:42:26,021] [INFO] Input size: 2048
[2016-09-09 19:42:26,022] [INFO] Output size: 288
[2016-09-09 19:42:26,025] [INFO] Hidden layer size: 1168
[2016-09-09 19:42:26,041] [INFO] Hidden layer size: 1168
[2016-09-09 19:42:26,069] [INFO] Hidden layer size: 1168


In [15]:
logger.info("Starting training phase.")
train(sess, model, args.num_epochs, train_dataset, args.batch_size) #, train_dataset, wv_model, test_dataset=test_dataset, epoch_verbosity_rate=100)

[2016-09-09 19:42:27,778] [INFO] Starting training phase.
[2016-09-09 19:43:00,889] [INFO] Finished epoch 0. (Avg. training loss: 148.447199865)
[2016-09-09 19:43:47,840] [INFO] Finished epoch 10. (Avg. training loss: 84.4973350005)
[2016-09-09 19:44:12,841] [INFO] Finished epoch 20. (Avg. training loss: 77.1428703395)
[2016-09-09 19:44:37,677] [INFO] Finished epoch 30. (Avg. training loss: 70.1581770724)
[2016-09-09 19:45:02,589] [INFO] Finished epoch 40. (Avg. training loss: 63.6083343679)
[2016-09-09 19:45:27,424] [INFO] Finished epoch 50. (Avg. training loss: 57.3654346466)
[2016-09-09 19:45:52,148] [INFO] Finished epoch 60. (Avg. training loss: 51.9774155834)
[2016-09-09 19:46:16,772] [INFO] Finished epoch 70. (Avg. training loss: 47.4211089394)
[2016-09-09 19:46:41,418] [INFO] Finished epoch 80. (Avg. training loss: 43.7574630434)
[2016-09-09 19:47:05,984] [INFO] Finished epoch 90. (Avg. training loss: 40.6061665362)
[2016-09-09 19:47:30,622] [INFO] Finished epoch 100. (Avg. trai

KeyboardInterrupt: 

In [16]:
logger.info("Starting evaluation phase.")
test_x, test_y = model.to_ndarrs(test_dataset)
predictions = model.predict(sess, test_x)
evaluator = Evaluation(test_y, predictions, k=5)
logger.info("Evaluation (precision, recall, f1): %s" % evaluator.evaluate())

[2016-09-09 19:54:02,409] [INFO] Starting evaluation phase.
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[2016-09-09 19:54:02,965] [INFO] Evaluation (precision, recall, f1): [0.40860336819079413, 0.28596257579458823, 0.31335328330840467]
